<a href="https://colab.research.google.com/github/nalgo-intern/team-b/blob/master/%E3%83%9D%E3%82%B1%E3%83%A2%E3%83%B3%E3%82%BF%E3%82%A4%E3%83%97%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#driveからデータを取得

In [0]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [0]:
!cp -R /content/drive/My\ Drive/dataset /content/
!mkdir learned_data

#水増し

In [0]:
#画像の水増し

import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

# 画像を拡張する関数
def draw_images(generator, x, dir_name, index):
    save_name = 'extened-' + str(index)
    g = generator.flow(x, batch_size=1, save_to_dir=output_dir,
                       save_prefix=save_name, save_format='jpg')

    # 1つの入力画像から何枚拡張するかを指定（今回は50枚）
    for i in range(50):
        bach = g.next()

types =["normal", "fire", "water", "grass", "electric", "ice", "fighting", "poison", "ground", "flying", "psychic", "bug", "rock", "ghost", "dragon"]
base_dir = './dataset/'
for i in types:
    print(i)
    # 出力先ディレクトリの設定
    output_dir = base_dir + i
    
    if not(os.path.exists(output_dir)):
        os.mkdir(output_dir)
    
    # 拡張する画像の読み込み
    images = glob.glob(os.path.join(base_dir + i, "*.jpg"))
    
    # ImageDataGeneratorを定義
    datagen = ImageDataGenerator(rotation_range=30,
                                width_shift_range=20,
                                height_shift_range=0.,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True)
    
    # 読み込んだ画像を順に拡張
    for i in range(len(images)):
        img = load_img(images[i])
        img = img.resize((250, 250))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        draw_images(datagen, x, output_dir, i)

Using TensorFlow backend.


normal
fire
water
grass
electric
ice
fighting
poison
ground
flying
psychic
bug
rock
ghost
dragon


#データセットを作成

In [0]:
!pip install -U numpy

     |████████████████████████████████| 20.4MB 2.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [0]:
from PIL import Image
import os, glob
import numpy as np
import random, math

root_dir = "./dataset"
types =["normal", "fire", "water", "grass", "electric", "ice", "fighting", "poison", "ground", "flying", "psychic", "bug", "rock", "ghost", "dragon"]

X = []
Y = []

def make_sample(files):
    global X, Y
    X = []
    Y = []
    for cat, fname in files:
        add_sample(cat, fname)
    return np.array(X), np.array(Y)


def add_sample(cat, fname):
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((120, 120))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

allfiles = []

for idx, cat in enumerate(types):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.8)
train = allfiles[0:th]
test = allfiles[th:]
X_train, y_train = make_sample(train)
np.save("./dataset_X_train.npy", X_train)
np.save("./dataset_y_train.npy", y_train)
X_test, y_test = make_sample(test)
np.save("./dataset_X_test.npy", X_test)
np.save("./dataset_y_test.npy", y_test)


#学習

In [0]:
!pip install numpy==1.16.2

     |████████████████████████████████| 17.3MB 2.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.1
    Uninstalling numpy-1.17.1:
      Successfully uninstalled numpy-1.17.1


In [0]:
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
import numpy as np

num_classes = 15
im_rows = 120
im_cols = 120
in_shape = (im_rows, im_cols, 3)

# データを読み込む --- (*1)
X_train = np.load("./dataset_X_train.npy")
y_train = np.load("./dataset_y_train.npy")
X_test = np.load("./dataset_X_test.npy")
y_test = np.load("./dataset_y_test.npy")

# データを正規化 --- (*2)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# ラベルデータをOne-Hot形式に変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## モデルを定義 --- (*3)
## 51行目のDenseの値を変えてみたり35-40行目を増やしたり減らしたり、Conv2Dの最初の引数を変えたり、Dropoutの引数を変えたり
#model = Sequential()
#model.add(Conv2D(32, (3, 3), padding='same',
#                 input_shape=in_shape))
#model.add(Activation('relu'))
#model.add(Conv2D(32, (3, 3)))
#model.add(Activation('relu'))
##model.add(MaxPooling2D(pool_size=(2, 2)))
##model.add(Dropout(0.5))
#
#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(Activation('relu'))
#model.add(Conv2D(128, (3, 3)))
#model.add(Activation('relu'))
##model.add(MaxPooling2D(pool_size=(2, 2)))
##model.add(Dropout(0.5))
#
##model.add(Conv2D(128, (3, 3), padding='same')
##model.add(Activation('relu'))
##model.add(Conv2D(128, (3, 3)))
##model.add(Activation('relu'))
##model.add(MaxPooling2D(pool_size=(2, 2)))
##model.add(Dropout(0.25))
#
#model.add(Flatten())
#model.add(Dense(512))
##model.add(Dense(128))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
#model.add(Dense(num_classes))
#model.add(Activation('sigmoid'))
#
## モデルをコンパイル --- (*4)
#model.compile(
#    loss='categorical_crossentropy',
#    optimizer='adam',
#    metrics=['accuracy'])

#綾鷹
model = Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(120,120,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.75))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512,activation="relu"))
model.add(Dense(15,activation="softmax"))
model.summary()
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=["acc"])

# 学習を実行 --- (*5)
#batch_sizeやepochsの数を変えたり
hist = model.fit(X_train, y_train,
    batch_size=128, epochs=200,
    verbose=1,
    validation_data=(X_test, y_test))

# モデルを評価 --- (*6)
score = model.evaluate(X_test, y_test, verbose=1)
print('正解率=', score[1], 'loss=', score[0])

# 学習の様子をグラフへ描画 --- (*7)
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#モデル・重みの保存
json_string = model.model.to_json()
open('./learned_data/data.json', 'w').write(json_string)

hdf5_file = "./learned_data/data.hdf5"
model.model.save_weights(hdf5_file)

Using TensorFlow backend.
W0827 01:12:01.125983 139919141156736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 01:12:01.176618 139919141156736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 01:12:01.206977 139919141156736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 01:12:01.247435 139919141156736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0827 01:12:01.267344 1399191

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 118, 118, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 128)       0         
__________

#バックアップ

In [0]:
!cp -R ./learned_data ./drive/My\ Drive
!cp dataset*.npy ./drive/My\ Drive